In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from transformers import pipeline
classifier = pipeline('sentiment-analysis')

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [ ]:
classifier("I hate you")

[{'label': 'NEGATIVE', 'score': 0.9991129040718079}]

In [ ]:
!pip install streamlit
!pip install pyngrok==4.1.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 74.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15963 sha256=4c0a78293e705bf704316c0a31f2eb1a7b3234026dfae2c32b68a3df82e53ffa
  Stored in directory: /root/.cache/pip/wheels/4c/7c/4c/632fba2ea8e88d8890102eb07bc922e1ca8fa14db5902c91a8
Successfully built pyngrok


In [ ]:
%%writefile app.py
import streamlit as st
from transformers import pipeline
model = pipeline('sentiment-analysis')
import random
import time
import re
import string
import pickle as pkl
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from keras.preprocessing.sequence import pad_sequences
import nltk
nltk.download('stopwords')
with open('/content/drive/MyDrive/sentiment_analysis_major_project/tokenizer.pkl', 'rb') as f:
    tokenizer = pkl.load(f)
#with open('/content/drive/MyDrive/sentiment_analysis_major_project/model.pkl', 'rb') as f:
  #  model = pkl.load(f)
stemmer = PorterStemmer()
stopwords = set(stopwords.words('english'))
def preprocess(text):
    text = str(text).lower()
    text = re.sub('https?://\S+|www\.\S+','', text)  # URL removal
    text = re.sub('<.*?>+', '', text)  # HTML tags removal
    text = re.sub('\[.*?\]', '', text)  # Square brackets removal
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)  # Punctuation removal
    text = re.sub('\n', '', text)  # Newline removal
    text = re.sub('\w*\d\w*', '', text)  # Words with digits removal
    text = ' '.join([word for word in text.split(' ') if word not in stopwords])  # Stopwords removal
    text = ' '.join([stemmer.stem(word) for word in text.split(' ')])  # Stemming
    sequence = tokenizer.texts_to_sequences([text])
    sequence = pad_sequences(sequence, maxlen=100)
    return sequence
def get_prediction(text):
  #text = preprocess(text)
    #if model.predict(text) >= 0.5:
        #return "Positive"
#else:
       # return "Negative"
    if model(text)[0]['label'] == 'POSITIVE':
        return "Positive"
    else:
        return "Negative"
def generate_text(prompt, length=100, temperature=0.5):
    response = client.generate_text(text=prompt, length=length, temperature=temperature)
    generated_text = response.text
    return generated_text
def get_bot_response(user_input):
    user_input_lower = user_input.lower()
    if "hello" in user_input_lower or "hai" in user_input_lower:
        return ("""Hi there! I am a Sentiment Analysis ChatBot developed by Sai Kiran Patnana..!\n
        I can help in doing Sentiment Analysis for your provided text..!\n
        Make sure you provide text starting with 'my text is....'!""")
    elif "how are you" in user_input_lower:
        return "I'm just a bot, but I'm doing well! How can I assist you today?"
    elif user_input_lower.startswith("my text is"):
        text_for_prediction = user_input[10:].strip()
        return f"Bingo..! The sentiment of your provided text is {get_prediction(text_for_prediction)}!"
    else:
        generated_text = generate_text(prompt=user_input, length=100)
        return generated_text
st.title("Sentiment Analysis Chatbot")
# Initialize chat history
if "messages" not in st.session_state:
    st.session_state.messages = []

# Display chat messages from history on app rerun
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])
# Accept user input
if prompt := st.chat_input("What is up?"):
    # Add user message to chat history
    st.session_state.messages.append({"role": "user", "content": prompt})
    # Display user message in chat message container
    with st.chat_message("user"):
        st.markdown(prompt)
    # Display assistant response in chat message container
    with st.chat_message("assistant"):
        message_placeholder = st.empty()
        full_response = ""
        assistant_response = get_bot_response(prompt)
        # Simulate stream of response with milliseconds delay
        for chunk in assistant_response.split():
            full_response += chunk + " "
            time.sleep(0.05)
            # Add a blinking cursor to simulate typing
            message_placeholder.markdown(full_response + "▌")
        message_placeholder.markdown(full_response)
    # Add assistant response to chat history
    st.session_state.messages.append({"role": "assistant", "content": full_response})

Writing app.py


In [1]:
# Run Streamlit in the background
get_ipython().system_raw('streamlit run app.py &')

from pyngrok import ngrok
# Set up the auth token
ngrok.set_auth_token("2ZwPANB2vsulJzM9iRbN0qFNeSn_66ULUFESMmu6xYVZhUNXM")

# Setup a tunnel to the streamlit port 8501
public_url = ngrok.connect(port='8501')
public_url

ModuleNotFoundError: ignored

In [ ]:
!pkill -f "streamlit"
from pyngrok import ngrok
ngrok.kill()
